## Step 1: Train your model

In [ ]:
!pip install garden-ai

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.03899014,  0.06659982, -0.610887  ,  1.287015  , -0.736691  ,
        -0.46360427, -0.8647197 , -0.48179212,  0.0216336 ,  0.4702802 ]],
      dtype=float32)

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.09416139, 0.09679738, 0.0491626 , 0.32800704, 0.04335097,
        0.0569638 , 0.03814141, 0.05593711, 0.09254119, 0.14493717]],
      dtype=float32)

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.8653393

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 10s 4ms/step - loss: 0.2942 - accuracy: 0.9150
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1419 - accuracy: 0.9589
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1081 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0869 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0741 - accuracy: 0.9763


In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 2s - loss: 0.0771 - accuracy: 0.9764 - 2s/epoch - 5ms/step


[0.07706169784069061, 0.9764000177383423]

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
preds = probability_model(x_test[:5])

In [18]:
tf.math.argmax(preds, 1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([7, 2, 1, 0, 4])>

In [17]:
preds

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.7616854e-07, 6.2676766e-09, 4.7820122e-06, 4.5317753e-05,
        3.6434050e-11, 4.7193970e-07, 1.4137489e-13, 9.9994588e-01,
        6.9334966e-08, 3.0771521e-06],
       [7.3700697e-08, 1.5193335e-05, 9.9997175e-01, 7.6355009e-06,
        6.1987843e-14, 3.7706261e-06, 8.6587093e-08, 4.0043649e-11,
        1.4844843e-06, 7.2564344e-14],
       [2.3644985e-08, 9.9444324e-01, 4.6003266e-04, 1.0505852e-05,
        1.9132623e-05, 2.3370717e-06, 1.8932791e-06, 4.8668669e-03,
        1.9581064e-04, 1.0519114e-07],
       [9.9948150e-01, 7.4989881e-09, 8.3552812e-05, 2.7498035e-07,
        2.3656143e-05, 1.9313375e-04, 1.7334154e-04, 3.2328811e-05,
        6.5988672e-08, 1.2127329e-05],
       [4.3425157e-06, 4.9900412e-10, 4.0176760e-06, 3.7101270e-08,
        9.8964429e-01, 7.9744552e-07, 7.3383071e-06, 1.8787132e-04,
        1.2788016e-06, 1.0149949e-02]], dtype=float32)>

## Step 2: Publish a pipeline that executes your model

In [20]:
model.save("/content")

In [ ]:
!zip -r /content/keras_model.zip /content

```garden-ai model register keras-example "/Users/will/Sandbox/seedlings/tf_pipeline/keras_model" tensorflow```


```
garden-ai pipeline register "/Users/will/Sandbox/seedlings/tf_pipeline/pipeline.py"
```

```
 garden-ai garden add-pipeline --garden 7b751dfe-e6c7-4993-8388-d68405582055 --pipeline 10.23677/0vak-kr28
```

```
garden-ai garden publish --garden 7b751dfe-e6c7-4993-8388-d68405582055
```

## Step 3: Execute your model remotely

In [27]:
from garden_ai import GardenClient
gc = GardenClient()
demo_garden = gc.get_published_garden("10.23677/gac2-7m91")

In [28]:
demo_garden.tf_pipeline(x_test[:5], endpoint="86a47061-f3d9-44f0-90dc-56ddc642c000")

Output()

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([7, 2, 1, 0, 4])>